In [1]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
def comment(url):
    wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver"
    driver = webdriver.Chrome(wd)
    driver.get(url)
    pages = 0
    try:
        while True:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"#cbox_module > div > div.u_cbox_paginate > a"))).click()
            time.sleep(1.5)
            print(pages, end=" ")
            pages+=1
    except exceptions.ElementNotVisibleException as e: # 페이지 끝
        pass
    
    except Exception as e: # 다른 예외 발생시 확인
        print(e)
    html = driver.page_source
    dom = BeautifulSoup(html, "lxml")
    # 댓글이 들어있는 페이지 전체 크롤링
    comments_raw = dom.find_all("span", {"class" : "u_cbox_contents"})
    # 댓글의 text만 뽑는다.
    comments = [comment.text for comment in comments_raw]
    return comments

In [3]:
p=comment('https://entertain.naver.com/read?oid=109&aid=0004166734')

Message: 



In [4]:
p

['ㅋㅋㅋㅋㅋ이 4명 케미 왜이리 웃김 ㅋㅋㅋㅋ 수다만 떨었는데도 빵빵터짐 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '의미부여없이 웃기기로는 놀면뭐하니에서 오늘이 젤웃겼음ㅋㅋ지석진 조세호냅두고선 문닫고도망치는거 최고의 킬링포인트ㅋㅋㅋㅋ',
 '유느 행복해하는거 넘 보기좋네요ㅎㅎㅎ',
 '오프닝으로 뽕뽑넼ㅋㅋㅋㅋㅋ개웃겨',
 '유느가 기뻐하면 좋아요😆😆😆😆 토크만으로도 웃김',
 '유재석 저렇게 즐거워 죽을 거 같은 얼굴로 미친듯이 웃는 거 근 5년 만에 처음 보는 듯. 나머지 셋 보고 웃는 건데 오히려 그 셋은 덤덤하고 혼자 웃겨서 뒤로 넘어가려는 게 너무 웃겨ㅋㅋㅋㅋㅋㅋㅋㅋ',
 '유재석 방송 아닌 진짜웃는것같아 보는사람도 조았어요. 그리고 실제로도 책마니 읽는다하네요..괜히 1인자가 아님',
 '런닝맨에서 수년을봤는데도 광수나오니까 뭔가신선함',
 '지석진 땀닦은 휴지보고 이광수가 살점떨어진줄 알았다고 한겈ㅋㅋㅋㅋㅋㅋㅋ진짜웃겨',
 '소중한 우리 산슬이가 타고있어요 ㅋㅋㅋㅋㅋㅋㅋ']

In [5]:
from tensorflow.keras.models import load_model
from konlpy.tag import Okt  
okt = Okt()

In [6]:
from selenium import webdriver
from selenium.common import exceptions
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
def model_load(comments):
    token_sent=[]
    for comment in comments:
        temp = okt.morphs(comment, stem=True)
        token_sent.append(temp)
        max_words = 35000
    tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
    tokenizer.fit_on_texts(token_sent) 
    token_sent = tokenizer.texts_to_sequences(token_sent)
    word_to_index = tokenizer.word_index
    vocab_size = len(word_to_index)+1
    max_len = 124
    X_data = pad_sequences(token_sent, maxlen=max_len)
    model = load_model('model11.h5')
    predict = model.predict_classes(X_data)
    for i in range(len(X_data)):
        if(predict[i] == 0):
            a=0
    return a

In [8]:
model_load(p)

0

In [9]:
from bs4 import BeautifulSoup
import urllib.request
import re

In [10]:
def get_text(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = ''
    for item in soup.find_all('div', id='articeBody'):
        text = text + str(item.find_all(text=True))
    return text

In [11]:
txt=get_text('https://entertain.naver.com/read?oid=109&aid=0004166734')

In [12]:
def clean_text(text):
    cleaned_text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\.]+", " ", text)
    return cleaned_text

In [13]:
clean = clean_text(txt)

In [14]:
def get_title(url):
    source_code_from_URL = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_URL, 'lxml', from_encoding='utf-8')
    text = soup.select_one('h2.end_tit')
    text_r = text.get_text()
    return text_r
title = get_title('https://entertain.naver.com/read?oid=109&aid=0004166734')
title

"\n\t\t\t'놀면 뭐하니' 유재석, 절친♥︎들과 남산→서점→방 탈출..포상휴가 '만끽'[종합]\n\t\t"

In [15]:
clean_text(title)

' 놀면 뭐하니 유재석 절친 들과 남산 서점 방 탈출..포상휴가 만끽 종합 '

In [21]:
from krwordrank.sentence import summarize_with_sentences

def keyword_sentence(text):#키워드 문장 추출
    data = text.split('.')
    keywords, sents = summarize_with_sentences(data, num_keywords=10, num_keysents=10)
    return sents

In [22]:
key = keyword_sentence(clean)
key

[' 놀면 뭐하니 방송화면 캡처 김보라 기자 유재석 지석진 이광수 조세호가 포상 휴가를 즐겼다',
 ' 조세호도 전문적으로 웃음을 드린다 고 소개했는데 유재석이 전문적인 거 치곤 조금 아쉽다 고 하자 조세호는 근근이 밥 벌어 먹고 산다 고 받아쳤다',
 ' 이광수의 의견에 따라 유재석은 남산 돈가스 집까지 걸어 가자고 했다',
 ' 조세호와 이광수도 평상시에 오는 건 좋은데 휴가에 여길 오냐 고 했다',
 ' 아내와 아이와 행복을 위해 달려나가고 있다 고 했고 이광수는 저는 개그맨은 아니지만 주말 예능을 하고 있다 고 말해 웃음을 안겼다',
 ' 세 사람은 유재석과 단둘이 대 로 떠나는 여행인 줄 알고 왔는데 알고 보니 인이 떠나는 여행이 됐다',
 ' 지석진은 이게 이 프로그램의 위엄이다',
 ' 이게 뭐냐 런닝맨 이냐 라고 비하했다',
 ' 걷는 걸 좋아한다 며 여행하면서 차 타는 것도 좋지만 걸으며 보는 걸 좋아한다 고 말했다',
 ' 이들은 아침 시에 연 남산 돈가스 집에서 아침식사를 했다']

In [23]:
key

[' 놀면 뭐하니 방송화면 캡처 김보라 기자 유재석 지석진 이광수 조세호가 포상 휴가를 즐겼다',
 ' 조세호도 전문적으로 웃음을 드린다 고 소개했는데 유재석이 전문적인 거 치곤 조금 아쉽다 고 하자 조세호는 근근이 밥 벌어 먹고 산다 고 받아쳤다',
 ' 이광수의 의견에 따라 유재석은 남산 돈가스 집까지 걸어 가자고 했다',
 ' 조세호와 이광수도 평상시에 오는 건 좋은데 휴가에 여길 오냐 고 했다',
 ' 아내와 아이와 행복을 위해 달려나가고 있다 고 했고 이광수는 저는 개그맨은 아니지만 주말 예능을 하고 있다 고 말해 웃음을 안겼다',
 ' 세 사람은 유재석과 단둘이 대 로 떠나는 여행인 줄 알고 왔는데 알고 보니 인이 떠나는 여행이 됐다',
 ' 지석진은 이게 이 프로그램의 위엄이다',
 ' 이게 뭐냐 런닝맨 이냐 라고 비하했다',
 ' 걷는 걸 좋아한다 며 여행하면서 차 타는 것도 좋지만 걸으며 보는 걸 좋아한다 고 말했다',
 ' 이들은 아침 시에 연 남산 돈가스 집에서 아침식사를 했다']